In [21]:
import requests
import json
import pandas as pd
from functools import reduce
import plotly.express as px

In [2]:
def api_to_df(url):
    """
    accesses api as json, creates dict, and then converts dict to pandas dataframe

    args:
        - url, string url for api access
    
    returns:
        - pandas dataframe with api data
    """
    response = requests.get(url).text
    doc = json.loads(response)


    values = doc["value"]
    dim_ids = doc["id"]
    dim_sizes = doc["size"]
    dimensions = doc["dimension"]

    dim_labels = {}
    for dim in dim_ids:
        dim_info = dimensions[dim]
        label_dict = dim_info["category"]["label"]
        index_dict = dim_info["category"]["index"]
        ordered_labels = [label_dict[key] for key, _ in sorted(index_dict.items(), key=lambda x: x[1])]
        dim_labels[dim] = ordered_labels

    def flat_index_to_coords(index, sizes):
        coords = []
        for size in reversed(sizes):
            coords.insert(0, index % size)
            index = index // size
        return coords

    records = []
    for flat_key, value in values.items():
        flat_index = int(flat_key)
        coord_indices = flat_index_to_coords(flat_index, dim_sizes)

        record = {}
        for i, dim in enumerate(dim_ids):
            record[dim] = dim_labels[dim][coord_indices[i]]
        record["value"] = value
        records.append(record)

    return pd.DataFrame(records)
 
    

In [3]:
def clean_df(df, indic, value, metadata_cols, unit = None, target_col = None):
    """
    takes in df as arg, converts indic to columns, adds unit to indic columns, makes value column part of indic columns,
      removes freq column, returns cleaned df

      args:
      - df: pandas df to be cleaned, df
      - indic: indicator column to be converted into multiple columns, string
      - unit: unit column to be combined into indic columns, string
      - value: value column to be turned into values in indic cols, string
      - metadata_cols: other cols to be accepted, list of strings
      - target_col: main string to be indic col, string
    """
        # Strip whitespace from column names
    df.columns = df.columns.str.strip()

    # Combine indicator and unit to form unique column names
    if unit:
        df["indic_clean"] = df[indic] + " [" + df[unit] + "]"
    else: 
        df["indic_clean"] = df[indic]

    # Pivot to wide format
    df_wide = df.pivot_table(
        index=metadata_cols,
        columns="indic_clean",
        values=value
    ).reset_index()

    # Flatten columns
    df_wide.columns.name = None

    # If a target_col is specified, return only that column + metadata
    if target_col:
        if target_col not in df_wide.columns:
            raise ValueError(
                f"Target column '{target_col}' not found. Available columns:\n{df_wide.columns.tolist()}"
            )
        return df_wide[metadata_cols + [target_col]]

    # If no target_col specified, return everything
    return df_wide

# Healthcare

In [4]:
healthcare = api_to_df("https://ec.europa.eu/eurostat/api/dissemination/statistics/1.0/data/hlth_hlye?format=JSON&time=2008&time=2009&time=2010&time=2011&time=2012&time=2013&time=2014&time=2015&time=2016&time=2017&time=2018&time=2019&time=2020&time=2021&time=2022&geo=BE&geo=BG&geo=CZ&geo=DK&geo=DE&geo=EE&geo=IE&geo=EL&geo=ES&geo=FR&geo=HR&geo=IT&geo=CY&geo=LV&geo=LT&geo=LU&geo=HU&geo=MT&geo=NL&geo=AT&geo=PL&geo=PT&geo=RO&geo=SI&geo=SK&geo=FI&geo=SE&unit=YR&unit=PC&sex=M&sex=F&indic_he=HLY_0&indic_he=HLY_PC_0&indic_he=LE_0&indic_he=HLY_50&indic_he=HLY_PC_50&indic_he=LE_50&indic_he=HLY_65&indic_he=HLY_PC_65&indic_he=LE_65&lang=en")
healthcare

,freq,unit,sex,indic_he,geo,time,value
0,Annual,Percentage,Females,Healthy life years at birth in percentage of t...,Austria,2008,71.9
1,Annual,Percentage,Females,Healthy life years at birth in percentage of t...,Austria,2009,73.1
2,Annual,Percentage,Females,Healthy life years at birth in percentage of t...,Austria,2010,72.8
3,Annual,Percentage,Females,Healthy life years at birth in percentage of t...,Austria,2011,71.7
4,Annual,Percentage,Females,Healthy life years at birth in percentage of t...,Austria,2012,74.7
...,...,...,...,...,...,...,...
7207,Annual,Year,Males,Life expectancy in absolute value at 65,Slovakia,2018,15.4
7208,Annual,Year,Males,Life expectancy in absolute value at 65,Slovakia,2019,15.7
7209,Annual,Year,Males,Life expectancy in absolute value at 65,Slovakia,2020,14.8
7210,Annual,Year,Males,Life expectancy in absolute value at 65,Slovakia,2021,13.3


In [5]:
healthcare_clean = clean_df(healthcare, "indic_he", "value", ["sex","geo","time"], "unit", "Healthy life years in absolute value at birth [Year]")
healthcare_clean

,sex,geo,time,Healthy life years in absolute value at birth [Year]
0,Females,Austria,2008,59.9
1,Females,Austria,2009,60.8
2,Females,Austria,2010,60.8
3,Females,Austria,2011,60.1
4,Females,Austria,2012,62.5
...,...,...,...,...
799,Males,Sweden,2018,73.7
800,Males,Sweden,2019,73.8
801,Males,Sweden,2020,72.8
802,Males,Sweden,2021,68.9


# Education

In [6]:
education = api_to_df("https://ec.europa.eu/eurostat/api/dissemination/statistics/1.0/data/edat_lfse_03?format=JSON&time=2008&time=2009&time=2010&time=2011&time=2012&time=2013&time=2014&time=2015&time=2016&time=2017&time=2018&time=2019&time=2020&time=2021&time=2022&geo=BE&geo=BG&geo=CZ&geo=DK&geo=DE&geo=EE&geo=IE&geo=EL&geo=ES&geo=FR&geo=HR&geo=IT&geo=CY&geo=LV&geo=LT&geo=LU&geo=HU&geo=MT&geo=NL&geo=AT&geo=PL&geo=PT&geo=RO&geo=SI&geo=SK&geo=FI&geo=SE&unit=PC&sex=M&sex=F&age=Y20-24&age=Y25-34&age=Y35-44&age=Y45-54&age=Y55-64&isced11=ED0-2&isced11=ED3_4&isced11=ED5-8&lang=en")
education

,freq,sex,age,unit,isced11,geo,time,value
0,Annual,Females,From 20 to 24 years,Percentage,"Less than primary, primary and lower secondary...",Austria,2008,15.2
1,Annual,Females,From 20 to 24 years,Percentage,"Less than primary, primary and lower secondary...",Austria,2009,14.0
2,Annual,Females,From 20 to 24 years,Percentage,"Less than primary, primary and lower secondary...",Austria,2010,13.9
3,Annual,Females,From 20 to 24 years,Percentage,"Less than primary, primary and lower secondary...",Austria,2011,13.4
4,Annual,Females,From 20 to 24 years,Percentage,"Less than primary, primary and lower secondary...",Austria,2012,13.0
...,...,...,...,...,...,...,...,...
12140,Annual,Males,From 55 to 64 years,Percentage,Tertiary education (levels 5-8),Slovakia,2018,16.0
12141,Annual,Males,From 55 to 64 years,Percentage,Tertiary education (levels 5-8),Slovakia,2019,15.8
12142,Annual,Males,From 55 to 64 years,Percentage,Tertiary education (levels 5-8),Slovakia,2020,17.8
12143,Annual,Males,From 55 to 64 years,Percentage,Tertiary education (levels 5-8),Slovakia,2021,18.0


In [7]:
education_clean = clean_df(education, "isced11", "value",["sex","age","geo","time"], "unit", target_col="Tertiary education (levels 5-8) [Percentage]")
education_clean

,sex,age,geo,time,Tertiary education (levels 5-8) [Percentage]
0,Females,From 20 to 24 years,Austria,2008,4.6
1,Females,From 20 to 24 years,Austria,2009,5.6
2,Females,From 20 to 24 years,Austria,2010,6.9
3,Females,From 20 to 24 years,Austria,2011,7.2
4,Females,From 20 to 24 years,Austria,2012,9.2
...,...,...,...,...,...
4045,Males,From 55 to 64 years,Sweden,2018,26.0
4046,Males,From 55 to 64 years,Sweden,2019,24.8
4047,Males,From 55 to 64 years,Sweden,2020,25.4
4048,Males,From 55 to 64 years,Sweden,2021,28.2


# Safety

In [8]:
safety = api_to_df("https://ec.europa.eu/eurostat/api/dissemination/statistics/1.0/data/crim_off_cat?format=JSON&time=2008&time=2009&time=2010&time=2011&time=2012&time=2013&time=2014&time=2015&time=2016&time=2017&time=2018&time=2019&time=2020&time=2021&time=2022&geo=BE&geo=BG&geo=CZ&geo=DK&geo=DE&geo=EE&geo=IE&geo=EL&geo=ES&geo=FR&geo=HR&geo=IT&geo=CY&geo=LV&geo=LT&geo=LU&geo=HU&geo=MT&geo=NL&geo=AT&geo=PL&geo=PT&geo=RO&geo=SI&geo=SK&geo=FI&geo=SE&unit=NR&unit=P_HTHAB&iccs=ICCS0101&iccs=ICCS0102&iccs=ICCS020111&iccs=ICCS020221&iccs=ICCS0301&iccs=ICCS03011&iccs=ICCS03012&iccs=ICCS0302&iccs=ICCS030221&iccs=ICCS0401&iccs=ICCS0501&iccs=ICCS05012&iccs=ICCS0502&iccs=ICCS05021&iccs=ICCS0601&iccs=ICCS0701&iccs=ICCS0703&iccs=ICCS07031&iccs=ICCS07041&iccs=ICCS0903&iccs=ICCS09051&lang=en")
safety

,freq,iccs,unit,geo,time,value
0,Annual,Intentional homicide,Number,Austria,2008,58.00
1,Annual,Intentional homicide,Number,Austria,2009,51.00
2,Annual,Intentional homicide,Number,Austria,2010,61.00
3,Annual,Intentional homicide,Number,Austria,2011,80.00
4,Annual,Intentional homicide,Number,Austria,2012,88.00
...,...,...,...,...,...,...
12681,Annual,Participation in an organized criminal group,Per hundred thousand inhabitants,Slovakia,2018,0.17
12682,Annual,Participation in an organized criminal group,Per hundred thousand inhabitants,Slovakia,2019,0.11
12683,Annual,Participation in an organized criminal group,Per hundred thousand inhabitants,Slovakia,2020,0.75
12684,Annual,Participation in an organized criminal group,Per hundred thousand inhabitants,Slovakia,2021,0.64


In [9]:
safety_clean = clean_df(safety, "iccs", "value",["geo","time"], "unit", target_col="Unlawful acts involving controlled drugs or precursors [Number]")
safety_clean

,geo,time,Unlawful acts involving controlled drugs or precursors [Number]
0,Austria,2008,1980.0
1,Austria,2009,2099.0
2,Austria,2010,2167.0
3,Austria,2011,2320.0
4,Austria,2012,1990.0
...,...,...,...
400,Sweden,2018,108685.0
401,Sweden,2019,115388.0
402,Sweden,2020,126245.0
403,Sweden,2021,120233.0


# Environmental Consciousness

In [10]:
environment = api_to_df("https://ec.europa.eu/eurostat/api/dissemination/statistics/1.0/data/sdg_13_10?format=JSON&time=2008&time=2009&time=2010&time=2011&time=2012&time=2013&time=2014&time=2015&time=2016&time=2017&time=2018&time=2019&time=2020&time=2021&time=2022&geo=BE&geo=BG&geo=CZ&geo=DK&geo=DE&geo=EE&geo=IE&geo=EL&geo=ES&geo=FR&geo=HR&geo=IT&geo=CY&geo=LV&geo=LT&geo=LU&geo=HU&geo=MT&geo=NL&geo=AT&geo=PL&geo=PT&geo=RO&geo=SI&geo=SK&geo=FI&geo=SE&unit=T_HAB&unit=I90&src_crf=TOTXMEMO&src_crf=TOTX4_MEMO&lang=en")
environment

,freq,src_crf,unit,geo,time,value
0,Annual,Total (excluding LULUCF and memo items),"Index, 1990=100",Austria,2008,109.9
1,Annual,Total (excluding LULUCF and memo items),"Index, 1990=100",Austria,2009,101.6
2,Annual,Total (excluding LULUCF and memo items),"Index, 1990=100",Austria,2010,107.3
3,Annual,Total (excluding LULUCF and memo items),"Index, 1990=100",Austria,2011,104.6
4,Annual,Total (excluding LULUCF and memo items),"Index, 1990=100",Austria,2012,101.1
...,...,...,...,...,...,...
1615,Annual,Total (excluding memo items),Tonnes per capita,Slovakia,2018,6.9
1616,Annual,Total (excluding memo items),Tonnes per capita,Slovakia,2019,6.4
1617,Annual,Total (excluding memo items),Tonnes per capita,Slovakia,2020,5.5
1618,Annual,Total (excluding memo items),Tonnes per capita,Slovakia,2021,6.2


In [11]:
env_clean = clean_df(environment, "src_crf", "value",["geo","time"], "unit", target_col="Total (excluding memo items) [Tonnes per capita]")
env_clean

,geo,time,Total (excluding memo items) [Tonnes per capita]
0,Austria,2008,9.3
1,Austria,2009,8.6
2,Austria,2010,8.8
3,Austria,2011,8.6
4,Austria,2012,8.5
...,...,...,...
400,Sweden,2018,1.1
401,Sweden,2019,1.0
402,Sweden,2020,0.6
403,Sweden,2021,1.1


# Public Infrastructure

In [12]:

infrastructure = api_to_df('https://ec.europa.eu/eurostat/api/dissemination/statistics/1.0/data/rail_pa_total?format=JSON&time=2008&time=2009&time=2010&time=2011&time=2012&time=2013&time=2014&time=2015&time=2016&time=2017&time=2018&time=2019&time=2020&time=2021&time=2022&geo=EU27_2020&geo=BE&geo=BG&geo=CZ&geo=DK&geo=DE&geo=EE&geo=IE&geo=EL&geo=ES&geo=FR&geo=HR&geo=IT&geo=LV&geo=LT&geo=LU&geo=HU&geo=NL&geo=AT&geo=PL&geo=PT&geo=RO&geo=SI&geo=SK&geo=FI&geo=SE&unit=MIO_PKM&unit=THS_PAS&lang=en')
infrastructure

,freq,unit,geo,time,value
0,Annual,Millions of passenger-kilometres,Austria,2008,10365
1,Annual,Millions of passenger-kilometres,Austria,2009,10184
2,Annual,Millions of passenger-kilometres,Austria,2010,10263
3,Annual,Millions of passenger-kilometres,Austria,2011,10778
4,Annual,Millions of passenger-kilometres,Austria,2012,11211
...,...,...,...,...,...
669,Annual,Thousand passengers,Slovakia,2018,77265
670,Annual,Thousand passengers,Slovakia,2019,80671
671,Annual,Thousand passengers,Slovakia,2020,49421
672,Annual,Thousand passengers,Slovakia,2021,45858


In [13]:
inf_clean = clean_df(infrastructure, "unit", "value",["geo","time"], target_col="Thousand passengers")
inf_clean

,geo,time,Thousand passengers
0,Austria,2008,236789.0
1,Austria,2009,237658.0
2,Austria,2010,239974.0
3,Austria,2011,241526.0
4,Austria,2012,260518.0
...,...,...,...
337,Sweden,2018,246490.0
338,Sweden,2019,264603.0
339,Sweden,2020,169163.0
340,Sweden,2021,164490.0


# Quality of Life Index

In [14]:

eu_countries = {
    "Austria", "Belgium", "Bulgaria", "Croatia", "Cyprus", "Czechia", "Denmark",
    "Estonia", "Finland", "France", "Germany", "Greece", "Hungary", "Ireland",
    "Italy", "Latvia", "Lithuania", "Luxembourg", "Malta", "Netherlands",
    "Poland", "Portugal", "Romania", "Slovakia", "Slovenia", "Spain", "Sweden"
}

df = pd.read_csv("qol_data.csv", encoding="ISO-8859-1")

df = df.dropna(axis=1, how='all')
df.columns = df.columns.str.strip()
df_filtered = df[["Year", "Country name", "Ladder score"]]

df_eu = df_filtered[df_filtered["Country name"].isin(eu_countries)]

df_eu = df_eu.reset_index(drop=True)

df_eu


,Year,Country name,Ladder score
0,2024.0,Finland,7.736
1,2024.0,Denmark,7.521
2,2024.0,Sweden,7.345
3,2024.0,Netherlands,7.306
4,2023.0,Austria,6.905
...,...,...,...
346,2016.0,Sweden,7.284
347,2015.0,Sweden,7.291
348,2014.0,Sweden,7.364
349,2012.0,Sweden,7.480


# Combining Dataframes

In [32]:
# standardizing columns
def prep_df(df, time_col="time", country_col="geo", new_cols={}):
    df = df.rename(columns={time_col: "year", country_col: "country", **new_cols})
    df["year"] = df["year"].astype(int)
    return df

healthcare = prep_df(healthcare_clean, new_cols={"Healthy life years in absolute value at birth [Year]": "healthcare"})
education = prep_df(education_clean, new_cols={"Tertiary education (levels 5-8) [Percentage]": "education"})
safety = prep_df(safety_clean, new_cols={"Unlawful acts involving controlled drugs or precursors [Number]": "safety"})
environment = prep_df(env_clean, new_cols={"Total (excluding memo items) [Tonnes per capita]": "environment"})
infrastructure = prep_df(inf_clean, new_cols={"Thousand passengers": "infrastructure"})
qol = prep_df(df_eu, time_col="Year", country_col="Country name", new_cols={"Ladder score": "qol"})

dfs = [healthcare, education, safety, environment, infrastructure, qol]
combined = reduce(lambda left, right: pd.merge(left, right, on=["country", "year"], how="outer"), dfs)

combined = combined.dropna(subset=["qol", "sex_x", "sex_y", "age", "education", "safety", "environment", "infrastructure"])
combined = combined.drop(columns=["sex_y"])
combined = combined.rename(columns={"sex_x": "sex"})

combined

,sex,country,year,healthcare,age,education,safety,environment,infrastructure,qol
60,Females,Austria,2011,60.1,From 20 to 24 years,7.2,2320.0,8.6,241526.0,7.227
61,Females,Austria,2011,60.1,From 25 to 34 years,22.4,2320.0,8.6,241526.0,7.227
62,Females,Austria,2011,60.1,From 35 to 44 years,19.5,2320.0,8.6,241526.0,7.227
63,Females,Austria,2011,60.1,From 45 to 54 years,14.9,2320.0,8.6,241526.0,7.227
64,Females,Austria,2011,60.1,From 55 to 64 years,11.4,2320.0,8.6,241526.0,7.227
...,...,...,...,...,...,...,...,...,...,...
8126,Males,Sweden,2022,67.5,From 20 to 24 years,10.5,115875.0,1.0,243972.0,7.395
8127,Males,Sweden,2022,67.5,From 25 to 34 years,44.3,115875.0,1.0,243972.0,7.395
8128,Males,Sweden,2022,67.5,From 35 to 44 years,49.3,115875.0,1.0,243972.0,7.395
8129,Males,Sweden,2022,67.5,From 45 to 54 years,41.1,115875.0,1.0,243972.0,7.395


# Plotting
Here, I want to plot quality of life against all other factors to see if there is any direct relationship between them that is immediately visible.

In [33]:
fig = px.scatter(
    combined,
    x='healthcare',
    y='qol',
    color='year',
    opacity=0.5,
    labels={'healthcare': 'Life Expectancy', 'qol': 'Quality of Life'},
    title='Comparing Relationship of Life Expectancy and Quality of Life in Europe',
    hover_data=['country', 'year', 'sex']
)

fig.show()


In [34]:
fig = px.scatter(
    combined,
    x='education',
    y='qol',
    color='year',
    opacity=0.5,
    labels={'education': 'Tertiary Education', 'qol': 'Quality of Life'},
    title='Comparing Relationship of Tertiary Education and Quality of Life in Europe',
    hover_data=['country', 'year', 'sex', 'age']
)

fig.show()

In [35]:
fig = px.scatter(
    combined,
    x='safety',
    y='qol',
    color='year',
    opacity=0.5,
    labels={'safety': 'Controlled Drug Crime', 'qol': 'Quality of Life'},
    title='Comparing Relationship of Controlled Drug Crime and Quality of Life in Europe',
    hover_data=['country', 'year']
)

fig.show()

In [36]:
fig = px.scatter(
    combined,
    x='environment',
    y='qol',
    color='year',
    opacity=0.5,
    labels={'environment': 'Emissions [Tonnes per capita]', 'qol': 'Quality of Life'},
    title='Comparing Relationship of Emissions and Quality of Life in Europe',
    hover_data=['country', 'year']
)

fig.show()

In [37]:
fig = px.scatter(
    combined,
    x='infrastructure',
    y='qol',
    color='year',
    opacity=0.5,
    labels={'infrastructure': 'Transit Passengers', 'qol': 'Quality of Life'},
    title='Comparing Relationship of Transit Passengers and Quality of Life in Europe',
    hover_data=['country', 'year']
)

fig.show()